In [28]:
import gensim
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
import os
import re
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import string

stop = stopwords.words('english') + list(string.punctuation)

In [29]:
data_folder = "data/ReligiousTexts/"
files = os.listdir(data_folder)

In [31]:
def clean_text(text):
    sents = []
    sentences = sent_tokenize(text)
    for sent in sentences:
        words = word_tokenize(sent)
        sent_words = [] 
        for word in words:
            if word not in stop and word.isalpha() and len(word)>3:
                sent_words.append(word)
        sents.append(sent_words)
    return sents
        
sents = []
for file in files:
    with open(data_folder+file,encoding="ISO-8859-1") as f:
        sents += clean_text(f.read().lower())

In [39]:
common_dictionary = Dictionary(sents)
common_corpus = [common_dictionary.doc2bow(text) for text in sents]

In [45]:
lda = gensim.models.LdaModel(common_corpus,id2word=common_dictionary ,num_topics=30)

In [46]:
print(sents[4])
lda.get_document_topics(common_dictionary.doc2bow(sents[0]))

['laughable', 'stories', 'collected']


[(10, 0.06888889),
 (12, 0.06888889),
 (16, 0.068888895),
 (17, 0.4688889),
 (20, 0.20222221),
 (25, 0.06888889)]

In [47]:
for t in lda.get_topic_terms(17, topn=100):
    print(lda.id2word[t[0]],t[1],t[0])

must 0.06300893 227
verse 0.051203456 2691
works 0.04658449 808
object 0.042710353 2321
view 0.042061675 535
case 0.040343214 2
meaning 0.032654304 1229
consciousness 0.032395262 6016
word 0.03223322 709
another 0.029717123 672
follows 0.023940617 1064
desire 0.022404723 3271
others 0.021321632 1057
rest 0.016872765 637
parts 0.013461753 1065
rather 0.01152361 1490
distinction 0.010431765 3310
however 0.009285379 413
worship 0.009251232 1611
sûtra 0.008723995 78030
depends 0.0072911154 10720
greater 0.006720978 1573
quoted 0.006504991 2926
members 0.0061430247 5424
reasoning 0.005849436 8171
conditions 0.005840492 8552
essentially 0.005029902 6944
single 0.0048367204 2736
short 0.0046855435 2322
depend 0.004583881 7209
wrong 0.0044618053 4838
probably 0.0044489256 888
remembrance 0.0042355917 1889
brings 0.0041811652 6202
keep 0.0040738205 1885
exalted 0.004066757 2339
translate 0.0040056193 3439
divinity 0.0039148233 5422
treasure 0.0036165663 3712
electronic 0.0035975175 5977
insight

In [23]:
for i in range(len(texts)):
    print(i,lda.get_document_topics(common_dictionary.doc2bow(texts[i])))

0 [(72, 0.7638529), (101, 0.23538104)]
1 [(72, 0.99968857)]
2 [(3, 0.11925234), (50, 0.060529023), (81, 0.547961), (112, 0.01878531), (114, 0.04459516), (124, 0.0930825), (126, 0.09072271)]
3 [(39, 0.048211783), (42, 0.070848875), (68, 0.23294333), (81, 0.4987581), (136, 0.09894859), (147, 0.014736211)]
4 [(81, 0.99811935)]
5 [(42, 0.44131184), (81, 0.5329123)]
6 [(4, 0.010387154), (25, 0.019198317), (50, 0.015236385), (59, 0.81495035), (81, 0.07525856), (126, 0.05849635)]
7 [(81, 0.025056476), (105, 0.96678454)]
8 [(39, 0.110371254), (42, 0.18652059), (72, 0.17729418), (81, 0.13611563), (105, 0.3725189), (124, 0.01202921)]
9 [(13, 0.15553361), (42, 0.01841621), (50, 0.024158591), (72, 0.03865001), (81, 0.42612925), (99, 0.19907753), (126, 0.12373887)]
10 [(39, 0.010975361), (42, 0.44475654), (81, 0.35598952), (99, 0.1786199)]
11 [(42, 0.011528565), (52, 0.016522838), (81, 0.96080846)]
12 [(42, 0.8029138), (81, 0.19498582)]
13 [(39, 0.027418114), (42, 0.702488), (81, 0.26328483)]
14 [(